In [1]:
import pandas as pd
import numpy as np

# Görselleştirme için
import matplotlib.pyplot as plt
import seaborn as sns

# Ayarlar
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")


In [2]:
# Ana maç verisi
df = pd.read_csv("train.csv")

# Hamle verisi (isteğe bağlı kullanılacak)
turns_df = pd.read_csv("turns.csv")

# İlk 5 satırı görüntüleyelim
df.head()


,game_id,nickname,score,rating
0,1,BetterBot,335,1637
1,1,stevy,429,1500
2,3,davidavid,440,1811
3,3,BetterBot,318,2071
4,4,Inandoutworker,119,1473


In [3]:
# Veri yapısını ve eksik değerleri kontrol edelim
df.info()

# Sayısal sütunların istatistikleri
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100820 entries, 0 to 100819
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   game_id   100820 non-null  int64 
 1   nickname  100820 non-null  object
 2   score     100820 non-null  int64 
 3   rating    100820 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 3.1+ MB


,game_id,score,rating
count,100820.000000,100820.000000,100820.000000
mean,36367.328605,399.029022,1875.592938
std,21020.288705,75.184778,231.791016
min,1.000000,-64.000000,1033.000000
25%,18139.000000,352.000000,1664.000000
50%,36339.500000,398.000000,1907.000000
75%,54625.000000,445.000000,2060.000000
max,72773.000000,1132.000000,2510.000000


Toplam Satır: 100.820

Sütunlar:

game_id – maç kimliği (aynı ID, aynı oyunu oynayan iki oyuncuya ait)

nickname – oyuncu adı (bot ya da insan)

score – oyuncunun bu oyundaki skoru

rating – oyuncunun bu oyundaki oyun öncesi derecesi (hedef değişken)

Dikkat Edilecek Noktalar:
game_id çiftler halinde tekrar eder (aynı oyunu oynayan iki kişi var).

score sütununda negatif değerler var (min: -64): Temizlenmeli veya nedenine bakılmalı.

nickname botları da içeriyor (BetterBot, STEEBot, HastyBot).

rating aralığı: 1033–2510

score aralığı: -64–1132 (potansiyel hatalar var).

 Şimdi Ne Yapacağız?
Skoru negatif olan verileri kontrol edeceğiz.

Oyuncular arasındaki maçları eşleştirmek için game_id gruplamasını kullanacağız.

Botlarla mı oynanmış? Ona göre is_bot_match gibi bir özellik oluşturabiliriz.

Özellik mühendisliği yapacağız (örneğin: skor farkı, kazanmış mı, toplam skor vb.).

In [7]:
# Negatif skorları olan satırları listele
df[df["score"] < 0]



,game_id,nickname,score,rating


Özellik Mühendisliği başlıyoruz:
Her oyunu 2 oyuncuya göre gruplayacağız.

Her oyuncu için:

score_diff (rakipten kaç puan fazla/altta aldı),

is_winner (kazandı mı?),

is_bot_match (rakibi bot mu?) gibi yeni sütunlar ekleyeceğiz.

In [6]:
# Negatif skor içeren satırı çıkar
df = df[df["score"] >= 0].reset_index(drop=True)


Özellik Mühendisliği başlıyoruz:
Her oyunu 2 oyuncuya göre gruplayacağız.

Her oyuncu için:

score_diff (rakipten kaç puan fazla/altta aldı),

is_winner (kazandı mı?),

is_bot_match (rakibi bot mu?) gibi yeni sütunlar ekleyeceğiz.

Aynı Oyunu Oynayan Oyuncuları Eşleştir

In [9]:
# Aynı game_id'ye sahip oyuncuları eşleştirerek aynı oyunu oynayan iki oyuncuyu yan yana getireceğiz
df_sorted = df.sort_values(by=['game_id', 'score'], ascending=[True, False])
df_sorted['player_index'] = df_sorted.groupby('game_id').cumcount()

# Geniş formatta iki oyuncuyu yan yana al
df_wide = df_sorted.pivot(index='game_id', columns='player_index', values=['nickname', 'score', 'rating']).dropna()

# Sütun adlarını düzenle
df_wide.columns = ['player1_nickname', 'player2_nickname', 'player1_score', 'player2_score', 'player1_rating', 'player2_rating']
df_wide = df_wide.reset_index()
df_wide.head()


,game_id,player1_nickname,player2_nickname,player1_score,player2_score,player1_rating,player2_rating
0,1,stevy,BetterBot,429,335,1500,1637
1,3,davidavid,BetterBot,440,318,1811,2071
2,4,BetterBot,Inandoutworker,478,119,1936,1473
3,5,STEEBot,stevy,427,325,1844,1500
4,6,STEEBot,HivinD,427,378,2143,2029


In [ ]:
Yeni Özellikleri Oluştur

In [10]:
# Skor farkı
df_wide["score_diff"] = df_wide["player1_score"] - df_wide["player2_score"]

# Kazanan oyuncu bilgisi
df_wide["player1_is_winner"] = df_wide["player1_score"] > df_wide["player2_score"]

# Bot karşılaşması bilgisi
bots = ["BetterBot", "STEEBot", "HastyBot"]
df_wide["player1_vs_bot"] = df_wide["player2_nickname"].isin(bots).astype(int)


Tahmin Edilecek Hedef ve Özellik Seti

In [11]:
# Modelin öğreneceği hedef: player1_rating
target = df_wide["player1_rating"]

# Özellikler
features = df_wide[[
    "player1_score",
    "player2_score",
    "score_diff",
    "player1_is_winner",
    "player1_vs_bot"
]]


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib


In [14]:
# Eğitim ve Test Ayrımı

In [15]:
# Veriyi ayır
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [16]:
# Random Forest modeli
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [20]:
from sklearn.metrics import mean_squared_error
import numpy as np

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.2f}")


RMSE: 215.77


In [21]:
# Modeli diske kaydet
joblib.dump(model, "model.pkl")


['model.pkl']